In [3]:
!pip install asyncpraw praw
!pip install --upgrade pip
!pip install yfinance --no-dependencies
!pip install multitasking --no-dependencies


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [4]:
from numpy.core.defchararray import rstrip

In [5]:
from google.colab import userdata
c_id = userdata.get('client_id')
c_secret = userdata.get('client_secret')
u_agent = userdata.get('user_agent')

In [6]:
import yfinance as yf
import time
import pandas as pd
import asyncpraw
from datetime import datetime
import praw
import nltk
nltk.download('vader_lexicon')
from textblob import TextBlob

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [1]:


reddit = praw.Reddit(
    client_id=c_id,
    client_secret=c_secret,
    user_agent=u_agent
)

print(reddit.read_only)
subreddit = reddit.subreddit("IndianStockMarket")

NameError: name 'praw' is not defined

In [40]:
top_posts = reddit.subreddit("IndianStockMarket").top(limit=None)

# Initialize a counter
post_count = 0

# Iterate through the posts and count them
for post in top_posts:
    post_count += 1

print(f"Total number of top posts: {post_count}")


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

Total number of top posts: 996


In [31]:


# Sentiment analysis function for TextBlob tool
def text_blob_sentiment(review, sub_entries_textblob):
    analysis = TextBlob(review)
    if analysis.sentiment.polarity >= 0.0001:
        if analysis.sentiment.polarity > 0:
            sub_entries_textblob['positive'] = sub_entries_textblob['positive'] + 1
            return 'Positive'

    elif analysis.sentiment.polarity <= -0.0001:
        if analysis.sentiment.polarity <= 0:
            sub_entries_textblob['negative'] = sub_entries_textblob['negative'] + 1
            return 'Negative'
    else:
        sub_entries_textblob['neutral'] = sub_entries_textblob['neutral'] + 1
        return 'Neutral'

In [32]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# create object for VADER sentiment function interaction
sia = SentimentIntensityAnalyzer()
def nltk_sentiment(review, sub_entries_nltk):
    vs = sia.polarity_scores(review)
    if not vs['neg'] > 0.05:
        if vs['pos'] - vs['neg'] > 0:
            sub_entries_nltk['positive'] = sub_entries_nltk['positive'] + 1
            return 'Positive'
        else:
            sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
            return 'Neutral'

    elif not vs['pos'] > 0.05:
        if vs['pos'] - vs['neg'] <= 0:
            sub_entries_nltk['negative'] = sub_entries_nltk['negative'] + 1
            return 'Negative'
        else:
            sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
            return 'Neutral'
    else:
        sub_entries_nltk['neutral'] = sub_entries_nltk['neutral'] + 1
        return 'Neutral'

In [33]:
def replies_of(top_level_comment, count_comment, sub_entries_textblob, sub_entries_nltk):
    global STSC
    if len(top_level_comment.replies) == 0:
        count_comment = 0
        return
    else:
        for num, comment in enumerate(top_level_comment.replies):
            try:
                count_comment += 1
                # print('-' * count_comment, comment.body)
                text_blob_sentiment(comment.body, sub_entries_textblob)
                nltk_sentiment(comment.body, sub_entries_nltk)

                # Search for stock ticker symbol in comment body and increment counter if found
                if comment.body.find(STS) != -1:
                   STSC += 1
            except:
                continue
            replies_of(comment, count_comment, sub_entries_textblob, sub_entries_nltk)

In [34]:
import time
global STSC
global STS
global company_keywords
global stock
STSC = 0
STS = 'NSE'
def main():
    global STS
    # Create an empty list to collect rows
    all_rows = []



    submissioncounter = 1
    for submission in top_posts:
        sub_entries_textblob = {'negative': 0, 'positive': 0, 'neutral': 0}
        sub_entries_nltk = {'negative': 0, 'positive': 0, 'neutral': 0}
        print('Submission:', submissioncounter, '   |  Title of the post:', submission.title)
        submissioncounter += 1

        # Analyze submission title sentiments
        text_blob_sentiment(submission.title, sub_entries_textblob)
        nltk_sentiment(submission.title, sub_entries_nltk)

        submission_comm = reddit.submission(id=submission.id)
        submission_comm.comments.replace_more(limit=None)  # Replace all "MoreComments" objects
        all_comments = submission_comm.comments.list()  # Get a list of all comments

        for count, top_level_comment in enumerate(submission_comm.comments):

            try:
                text_blob_sentiment(top_level_comment.body, sub_entries_textblob)
                nltk_sentiment(top_level_comment.body, sub_entries_nltk)
                replies_of(top_level_comment, count, sub_entries_textblob, sub_entries_nltk)
            except:
                continue

        unix_time = submission_comm.created_utc
        commentPostedUnixTime = str(datetime.fromtimestamp(unix_time))
        commentYMD = commentPostedUnixTime[0:11].strip()

        # Get the sentiment values
        textblobnegative = sub_entries_textblob.get('negative', 0)
        textblobpositive = sub_entries_textblob.get('positive', 0)
        textblobneutral = sub_entries_textblob.get('neutral', 0)

        vadernegative = sub_entries_nltk.get('negative', 0)
        vaderpositive = sub_entries_nltk.get('positive', 0)
        vaderneutral = sub_entries_nltk.get('neutral', 0)

        # If there are mentions of the stock ticker, append data to the list
        if STSC > 0:
            commentYMD = datetime.strptime(commentYMD, '%Y-%m-%d')
            rowsToAppend = {
                'Title': submission.title, 'Ticker': STS, 'Date': commentYMD,
                'NumberOfTickerMentions': STSC, 'Vader Neg': vadernegative,
                'Vader Pos': vaderpositive, 'Vader Neut': vaderneutral,
                'textblob Negative': textblobnegative, 'textblob Positive': textblobpositive,
                'textblob Neut': textblobneutral
            }
            all_rows.append(rowsToAppend)
    print(all_rows)
    # Create DataFrame from the collected rows
    commentDF = pd.DataFrame(all_rows)
    filename = f'comment_analysis_{STS}.csv'
    commentDF.to_csv(filename, index=False, header=True)


    print('\nSentiment analysis is below:\n')
    print(commentDF)
    print('\nProcess complete. Files have been saved.')


In [35]:
if __name__ == '__main__':
  main()

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Submission: 1    |  Title of the post: But when will my LIC stocks 10x?


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Submission: 2    |  Title of the post: Meme just got real☠️


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

Submission: 994    |  Title of the post: Experts ne toh value trap bola tha😯


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Submission: 995    |  Title of the post: Thoughts?


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Submission: 996    |  Title of the post: Koi nazar nahi lagayega 
[{'Title': 'Thoughts?', 'Ticker': 'NSE', 'Date': datetime.datetime(2023, 10, 30, 0, 0), 'NumberOfTickerMentions': 1, 'Vader Neg': 76, 'Vader Pos': 140, 'Vader Neut': 264, 'textblob Negative': 73, 'textblob Positive': 218, 'textblob Neut': 189}, {'Title': 'You win some, you lose some. When you do win, make sure it’s big.', 'Ticker': 'NSE', 'Date': datetime.datetime(2023, 11, 30, 0, 0), 'NumberOfTickerMentions': 1, 'Vader Neg': 12, 'Vader Pos': 68, 'Vader Neut': 71, 'textblob Negative': 14, 'textblob Positive': 86, 'textblob Neut': 51}, {'Title': 'Should I buy and hold for a year?', 'Ticker': 'NSE', 'Date': datetime.datetime(2023, 9, 14, 0, 0), 'NumberOfTickerMentions': 1, 'Vader Neg': 21, 'Vader Pos': 14, 'Vader Neut': 53, 'textblob Negative': 17, 'textblob Positive': 19, 'textblob Neut': 52}, {'Title': 'For newbies here, -10% is correction and -30% is called market crash. So, market is no where near crash. Stop panicking

In [2]:
import yfinance as yf
import pandas as pd

def fetch_and_save_stock_history_with_pcr(ticker):
    # Get the stock data for the provided ticker
    selectedTicker = yf.Ticker(ticker)
    hist = selectedTicker.history(period="max")


    # Print the historical stock data (optional)
    print('\nStock History:\n', hist.head(), '\n')


    # Save the stock history and PCR data to CSV files with ticker in the filename
    hist.to_csv(f'stockhistory_{ticker}.csv', index=True, header=True)


    print(f'\nStock history and PCR data have been saved to stockhistory_{ticker}.csv and stockpcr_{ticker}.csv.')

# Example usage
STS = "ZOMATO.NS"
# Replace with your stock ticker
fetch_and_save_stock_history_with_pcr(STS)



Stock History:
                                  Open        High         Low       Close  \
Date                                                                        
2021-07-23 00:00:00+05:30  116.000000  138.899994  115.000000  126.000000   
2021-07-26 00:00:00+05:30  126.349998  143.750000  125.300003  140.649994   
2021-07-27 00:00:00+05:30  141.699997  147.800003  127.750000  132.899994   
2021-07-28 00:00:00+05:30  131.000000  135.000000  123.550003  131.199997   
2021-07-29 00:00:00+05:30  134.949997  144.000000  132.199997  141.550003   

                              Volume  Dividends  Stock Splits  
Date                                                           
2021-07-23 00:00:00+05:30  694895290        0.0           0.0  
2021-07-26 00:00:00+05:30  249723854        0.0           0.0  
2021-07-27 00:00:00+05:30  240341900        0.0           0.0  
2021-07-28 00:00:00+05:30  159793731        0.0           0.0  
2021-07-29 00:00:00+05:30  117973089        0.0           0